In [2]:
# Load the input
with open("day1_input.txt") as f:
    day1_input = f.read()

In [3]:
# Part 1: brute force
line_vals = []
for line in day1_input.split("\n"):
    first_numeric = last_numeric = None
    for char_idx in range(len(line)):
        if first_numeric is None and line[char_idx].isnumeric():
            first_numeric = line[char_idx]
        if last_numeric is None and line[-(char_idx+1)].isnumeric():
            last_numeric = line[-(char_idx+1)]
    
    line_val = int(first_numeric + last_numeric)
    line_vals.append(line_val)

In [ ]:
print(f"Answer (part one): {sum(line_vals)}")

In [5]:
# Part 2 - data
nums_as_strings = {
    "one":   1,
    "two":   2,
    "three": 3,
    "four":  4,
    "five":  5,
    "six":   6,
    "seven": 7,
    "eight": 8,
    "nine":  9,
}
max_num_len = max([len(x) for x in nums_as_strings.keys()])

# Array of sets, set at idx 0 contains the valid 1 char starts, 1 the 2 chars etc.
valid_starts = []
valid_ends = []
for i in range(max_num_len+1):
    valid_starts.append(set([nas[0:i] if len(nas) >= i else None for nas in nums_as_strings.keys()]))
    valid_ends.append(set([nas[len(nas)-i:] if len(nas) >= i else None for nas in nums_as_strings.keys()]))

In [6]:
# Left to right iterator test
# i is the marker for the left hand side of our search frame, j is the right
# if i == j then the character at target[i] is what we're looking at
# i can never be greater than j
# for every slice target[i:i] to target[i:j], we should check if that slice represents a number
# if the slice target[i:n] is not a valid start for that n (i.e. n=2 and target[1:n] is a 2 char
# string which isn't a valid start of a word-number) then we move i up by 1

# Reverse mode is the same logic but we move the slice back along the string from the end
def get_first_num_in_string(target: str, reverse=False) -> int:
    target_len = len(target)
    def move_marker(x):
        return x + 1
    def check_marker(x):
        return x < target_len
    def reset_i():
        return 0
    def reset_j(i):
        return i + 1
    def get_frame(i,j):
        if reverse:
            return target[target_len-j:target_len-i]
        else:
            return target[i:j]
    def check_frame_validity(frame):
        if reverse:
            return frame in valid_ends[frame_size]
        else:
            return frame in valid_starts[frame_size]

        
    i = reset_i()
    
    result = None
    # Expecting one, eight
    # Don't forget to check actual numerics!
    while check_marker(i) and result is None:
        j = reset_j(i)

        # Check for an actual number
        frame = get_frame(i,j)
        if frame.isnumeric():
            result = int(frame)
            break

        while check_marker(j):
            frame_size = j - i
            frame = get_frame(i,j)
            if not check_frame_validity(frame):
                break

            if frame in nums_as_strings:
                result = nums_as_strings[frame]
                break

            j = move_marker(j)
        
        i = move_marker(i)
    
    return result

In [ ]:
# Part 2
line_vals = []
for line in day1_input.split("\n"):
    first_number = get_first_num_in_string(target = line,reverse=False)
    last_number = get_first_num_in_string(target = line,reverse=True)

    if first_number is None or last_number is None:
        raise Exception(f"Found no numbers in the string! ({line})")
    
    line_val = first_number*10 + last_number
    line_vals.append(line_val)

print(f"Answer (part two): {sum(line_vals)}")